# VGG + Deep Clustering

In [1]:
# ! pip install torch torchvision numpy pydantic umap-learn interpret faiss-cpu torchsummary matplotlib pandas seaborn scikit-learn scikit-image pytorch-lightning[extra] --no-cache-dir --ignore-installed

In [1]:
import copy
import numpy as np
import os
import pickle
import torch
import torch.nn as nn
import torch.optim as optim

from data.brain_tumor_mri import BrainTumorMRI
from models.vgg_deep import DVGG
from pytorch_lightning import Trainer
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from pytorch_lightning.loggers import CSVLogger
from torchvision import transforms
from torchvision.transforms import functional as F

In [2]:
RND_SEED=31
torch.manual_seed(RND_SEED)
torch.cuda.manual_seed_all(RND_SEED)
np.random.seed(RND_SEED)

CUDA_AVAILABLE = torch.cuda.is_available()
DEVICE = torch.device("cpu:0")
BATCH_SIZE = 64
NUM_WORKERS = 4
EPOCHS = 30
DATASETS_PATH = './datasets/'
DATASET_RATIO = None

In [3]:
brain_tumor = BrainTumorMRI(DATASETS_PATH, train_size=0.7, val_size=0.3, random_state=555, transforms=transforms.Compose(
            [
                transforms.Resize((255, 255)),
                transforms.RandomHorizontalFlip(),
                transforms.RandomRotation(10),
                transforms.ToTensor(),
                transforms.Normalize(
                    mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]
                ),
            ]
        ))

In [4]:
model = DVGG(dataset=brain_tumor, num_classes=4, freeze=False)

logger = CSVLogger(save_dir="./", name="experiments/dvgg", flush_logs_every_n_steps=1)
checkpoint_callback = ModelCheckpoint(
    filename="dvgg-{epoch}-{train_loss:.4f}-{train_acc:.4f}-{val_loss:.4f}-{val_acc:.4f}",
    save_top_k=3, save_last=True, monitor="val_loss", mode="min", verbose=True
)
early_stopping_callback = EarlyStopping(monitor="val_loss", patience=3, mode="min", verbose=True)

trainer = Trainer(
    max_epochs=EPOCHS,
    callbacks=[checkpoint_callback],
    log_every_n_steps=1,
    logger=logger,
    limit_train_batches=DATASET_RATIO or 1.0,
    limit_val_batches=DATASET_RATIO or 1.0,
    limit_test_batches=DATASET_RATIO or 1.0
)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
`Trainer(limit_train_batches=1.0)` was configured so 100% of the batches per epoch will be used..
`Trainer(limit_val_batches=1.0)` was configured so 100% of the batches will be used..
`Trainer(limit_test_batches=1.0)` was configured so 100% of the batches will be used..


## TRAINING

In [ ]:
trainer.fit(
    model,
    train_dataloaders=brain_tumor.train_dataloader,
    val_dataloaders=brain_tumor.val_dataloader
)


  | Name      | Type               | Params
-------------------------------------------------
0 | model     | VGG                | 134 M 
1 | loss_fn   | CrossEntropyLoss   | 0     
2 | train_acc | MulticlassAccuracy | 0     
3 | val_acc   | MulticlassAccuracy | 0     
4 | test_acc  | MulticlassAccuracy | 0     
-------------------------------------------------
134 M     Trainable params
0         Non-trainable params
134 M     Total params
537.108   Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Epoch 0
Feature extracting elapsed time: 979.5813 seconds.


/Users/arthuralvim/Library/Caches/pypoetry/virtualenvs/xai-clustering-b3gHD5dt-py3.9/lib/python3.9/site-packages/threadpoolctl.py:1019: RuntimeWarning: libc not found. The ctypes module in Python 3.9 is maybe too old for this OS.
  warnings.warn(


...

Epoch 0, global step 125: 'val_loss' reached 0.17417 (best 0.17417), saving model to './experiments/dvgg/version_0/checkpoints/dvgg-epoch=0-train_loss=0.3786-train_acc=0.8552-val_loss=0.1742-val_acc=0.9329.ckpt' as top 3


Epoch 1
Creating dataset from Clustering elapsed time: 40.5298 seconds.
Feature extracting elapsed time: 934.8795 seconds.
Clustering elapsed time: 9.9143 seconds.
Metrics: {'clu_homogeneity_score': 0.0011255322807178414, 'clu_completeness_score': 0.0011106862014119456, 'clu_v_measure_score': 0.001118059960086197, 'clu_adjusted_rand_score': 0.00043790698964923377, 'clu_adjusted_mutual_info_score': 0.00029038620663324947}


Validation: 0it [00:00, ?it/s]

Epoch 1, global step 250: 'val_loss' reached 0.11978 (best 0.11978), saving model to './experiments/dvgg/version_0/checkpoints/dvgg-epoch=1-train_loss=0.2363-train_acc=0.9270-val_loss=0.1198-val_acc=0.9574.ckpt' as top 3


Epoch 2
Creating dataset from Clustering elapsed time: 41.5208 seconds.
Feature extracting elapsed time: 986.5664 seconds.
Clustering elapsed time: 13.1650 seconds.
Metrics: {'clu_homogeneity_score': 0.7729432430002073, 'clu_completeness_score': 0.7766884991392877, 'clu_v_measure_score': 0.7748113451739602, 'clu_adjusted_rand_score': 0.7501870471122439, 'clu_adjusted_mutual_info_score': 0.7746255456193114}


In [18]:
with open(f"{trainer.logger.log_dir}/dvgg-timing.dump", "wb") as f:
    pickle.dump(model.timers, f)

In [21]:
with open(f"{trainer.logger.log_dir}/dvgg-original-targets.dump", "wb") as f:
    pickle.dump(model.original_train_targets, f)

## TESTING

In [12]:
batch_preds = trainer.predict(model, dataloaders=brain_tumor.test_dataloader)

Predicting: 0it [00:00, ?it/s]

In [13]:
truth_labels = []
preds_labels = []
for truth, pred in batch_preds:
    truth_labels.append(truth)
    preds_labels.append(pred)
truth_labels = np.concatenate(truth_labels)
preds_labels = np.concatenate(preds_labels)

In [14]:
with open(f"{trainer.logger.log_dir}/dvgg-preds.dump", "wb") as f:
    pickle.dump([truth_labels, preds_labels], f)